<a href="https://colab.research.google.com/github/neohack22/IASD/blob/recommandation/recommendation/demos/1_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommender Systems Project

## 0. Quick Start
To run this notebook you just need to have [pipenv](https://github.com/pypa/pipenv) installed.
Then run these 3 commands:
- First install the dependencies with: `pipenv install`
- Launch the virtual env: `pipenv shell`
- Finally start jupyter and open the notebook: `jupyter-lab`

In [ ]:
%%bash
pip install surprise

  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617646 sha256=3ad4d785ae4ef38801e71c3ebde5b9ae956542673331da78aec15b975883c972
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [1]:
%%bash
pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633986 sha256=72e7fd852666b7bd4e934f8a14e21954c084becdb0a2e3309d592235b853eae8
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel 

from surprise import NormalPredictor, SVD, KNNBasic, NMF
from surprise import Dataset, Reader
from surprise import accuracy
from surprise.model_selection import cross_validate, KFold

In [2]:
from tqdm import tqdm
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import linear_kernel

from surprise import NormalPredictor, SVD, KNNBasic, NMF
from surprise import Dataset, Reader
from surprise import accuracy
from surprise.model_selection import cross_validate, KFold

## 1. Introduction
Recommender systems goal is to push *relevant* items to a given user. Understanding and modelling the user's preferences is required to reach this goal. In this project you will learn how to model the user's preferences with the [Surprise library](http://surpriselib.com/) to build different recommender systems. The first one will be a pure *collaborative filtering* approach, and the second one will rely on item attributes in a *content-based* way.

## 2. Loading Data
We use here the [MovieLens dataset](https://grouplens.org/datasets/movielens/). It contains 25 millions of users ratings. the data are in the `./data/raw` folder. We could load directly the .csv file with [a built-in Surprise function](https://github.com/NicolasHug/Surprise/blob/ef3ed6e98304dbf8d033c8eee741294b05b5ba07/surprise/dataset.py#L105), but it's more convenient to load it through a Pandas dataframe for later flexibility purpose.

In [ ]:
RATINGS_DATA_FILE = './data/raw/ratings.csv'
MOVIES_DATA_FILE = './data/raw/movies.csv'

In [ ]:
# load the raw csv into a data_frame
df_ratings = pd.read_csv(RATINGS_DATA_FILE)

# drop the timestamp column since we dont need it now
df_ratings = df_ratings.drop(columns="timestamp")

# movies dataframe
df_movies = pd.read_csv(MOVIES_DATA_FILE)

In [ ]:
# check we have 25M users' ratings
df_ratings.userId.count()

25000095

In [ ]:
def get_subset(df, number):
    """
        just get a subset of a large dataset for debug purpose
    """
    rids = np.arange(df.shape[0])
    np.random.shuffle(rids)
    df_subset = df.iloc[rids[:number], :].copy()
    return df_subset
df_ratings_100k = get_subset(df_ratings, 100000)
df_movies_100 = get_subset(df_movies, 100)

In [ ]:
# Surprise reader
reader = Reader(rating_scale=(0, 5))

# Finally load all ratings
ratings = Dataset.load_from_df(df_ratings_100k, reader)

## 3. Collaborative Filtering
We can test first any of the [Surprise algorithms](https://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html).

In [ ]:
# define a cross-validation iterator
kf = KFold(n_splits=3)

algos = [SVD(), NMF(), KNNBasic()]    

In [ ]:
def get_rmse(algo, testset):
        predictions = algo.test(testset)
        accuracy.rmse(predictions, verbose=True)
        
for trainset, testset in tqdm(kf.split(ratings)): 
    """
        get an evaluation with cross-validation for different algorithms
    """  
    for algo in algos:
        algo.fit(trainset)
        get_rmse(algo, testset)

0it [00:00, ?it/s]

RMSE: 1.0408
RMSE: 1.0949
Computing the msd similarity matrix...


1it [00:01,  1.57s/it]

Done computing similarity matrix.
RMSE: 1.0608
RMSE: 1.0489
RMSE: 1.1087
Computing the msd similarity matrix...


2it [00:03,  1.56s/it]

Done computing similarity matrix.
RMSE: 1.0745
RMSE: 1.0303
RMSE: 1.0855
Computing the msd similarity matrix...


3it [00:04,  1.54s/it]

Done computing similarity matrix.
RMSE: 1.0559


## 4. Content-based Filtering
Here we will rely directly on items attributes. First we have to describe a user profile with an attributes vector. Then we will use these vectors to generate recommendations.

In [ ]:
# computing similarities requires too much ressources on the whole dataset, so we take the subset with 100 items
df_movies_100 = df_movies_100.reset_index(drop=True)
df_movies_100.head(5)

,movieId,title,genres
0,162126,Autobiography of a Princess (1975),(no genres listed)
1,194666,Roads in February (2018),Drama
2,157679,Alley Cats Strike (2000),Children|Comedy|Drama
3,169196,Once Upon a Time Veronica (2012),Drama
4,191777,Revenge: A Love Story (2010),Thriller


In [ ]:
# we compute a TFIDF on the titles of the movies
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df_movies_100['title'])

In [ ]:
# we get cosine similarities: this takes a lot of time on the real dataset
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
# we generate in 'results' the most similar movies for each movie: we put a pair (score, movie_id)
results = {}
for idx, row in df_movies_100.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1] 
    similar_items = [(cosine_similarities[idx][i], df_movies_100['movieId'].loc[[i]].tolist()[0]) for i in similar_indices] 
    results[idx] = similar_items[1:]

In [ ]:
len(results)

100

In [ ]:
# transform a 'movieId' into its corresponding movie title
def item(id):  
    return df_movies_100.loc[df_movies_100['movieId'] == id]['title'].tolist()[0].split(' - ')[0] 

In [ ]:
# transform a 'movieId' into the index id
def get_idx(id):
    return df_movies_100[df_movies_100['movieId'] == id].index.tolist()[0]

In [ ]:
# Finally we put everything together here:
def recommend(item_id, num):
    print("Recommending " + str(num) + " products similar to " + item(item_id) + "...")   
    print("-------")    
    recs = results[get_idx(item_id)][:num]   
    for rec in recs: 
        print("\tRecommended: " + item(rec[1]) + " (score:" +      str(rec[0]) + ")")

Suppose a user wants the 10 most 'similar' (from a CBF point of view) movies from the movie 'Alley Cats Strike':

In [ ]:
recommend(item_id=157679, num=10)

Recommending 10 products similar to Alley Cats Strike (2000)...
-------
	Recommended: Ringu 0: Bâsudei (2000) (score:0.10424703060511913)
	Recommended: 6th Day, The (2000) (score:0.10424703060511913)
	Recommended: Room 205 of Fear (2011) (score:0.0)
	Recommended: Legend (2015) (score:0.0)
	Recommended: Hardcore (2001) (score:0.0)
	Recommended: The Huntress: Rune of the Dead (2019) (score:0.0)
	Recommended: House of Dracula (1945) (score:0.0)
	Recommended: Schramm (1993) (score:0.0)
	Recommended: The Coed and the Zombie Stoner (2014) (score:0.0)
	Recommended: Honor Among Lovers (1931) (score:0.0)
